# 1 - IMPORTS

In [184]:
import pandas as pd
import plotly.express as px
from haversine import haversine

# 2 - Data

In [161]:
data = pd.read_csv('data/uber.csv')
df = data.copy()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         200000 non-null  int64  
 1   key                200000 non-null  object 
 2   fare_amount        200000 non-null  float64
 3   pickup_datetime    200000 non-null  object 
 4   pickup_longitude   200000 non-null  float64
 5   pickup_latitude    200000 non-null  float64
 6   dropoff_longitude  199999 non-null  float64
 7   dropoff_latitude   199999 non-null  float64
 8   passenger_count    200000 non-null  int64  
dtypes: float64(5), int64(2), object(2)
memory usage: 13.7+ MB


In [162]:
data.head()

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


In [163]:
# Corrida não finalizada, sem passageiros
data[data['dropoff_latitude'].isna()]

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
87946,32736015,2013-07-02 03:51:57.0000001,24.1,2013-07-02 03:51:57 UTC,-73.950581,40.779692,NaN,NaN,0


In [164]:
# A coluna 'key' pode ser desconsiderada, contém os mesmos dados de 'pickup_datetime', e já temos uma coluna com o id das corridas
data['Unnamed: 0'].nunique(), data['key'].nunique()

(200000, 200000)

# 3 - Data Transform

In [165]:
# Drop da coluna 'key'
df.drop(columns='key', axis=1, inplace=True)
# Drop da corrida não finalizada
df.dropna(inplace=True)

# Conversão da coluna da data para o formato datetime e criação das colunas de ano, mês e dia,
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['year'] = df['pickup_datetime'].apply(lambda x: x.year)
df['month'] = df['pickup_datetime'].apply(lambda x: x.month)
df['day'] = df['pickup_datetime'].apply(lambda x: x.day)

# Renomeando as colunas 'Unnamed: 0 ' e 'fare_amount'
new_cols = ['id',
 'fare',
 'pickup_date',
 'pickup_longitude',
 'pickup_latitude',
 'dropoff_longitude',
 'dropoff_latitude',
 'passenger_count',
 'year',
 'month',
 'day']

df.columns = new_cols

In [166]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 199999 entries, 0 to 199999
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   id                 199999 non-null  int64              
 1   fare               199999 non-null  float64            
 2   pickup_date        199999 non-null  datetime64[ns, UTC]
 3   pickup_longitude   199999 non-null  float64            
 4   pickup_latitude    199999 non-null  float64            
 5   dropoff_longitude  199999 non-null  float64            
 6   dropoff_latitude   199999 non-null  float64            
 7   passenger_count    199999 non-null  int64              
 8   year               199999 non-null  int64              
 9   month              199999 non-null  int64              
 10  day                199999 non-null  int64              
dtypes: datetime64[ns, UTC](1), float64(5), int64(5)
memory usage: 18.3 MB


# 4 - Explore

## 4.1 - Passengers

In [167]:
df['passenger_count'].value_counts()

passenger_count
1      138425
2       29428
5       14009
3        8881
4        4276
6        4271
0         708
208         1
Name: count, dtype: int64

708 corridas não tiveram passageiros, 1 corrida teve 208 passageiros

In [168]:
# Corrida com 208 passageiros
df[df['passenger_count'] == 208]

,id,fare,pickup_date,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day
113038,35893772,11.7,2010-12-28 08:20:00+00:00,-73.937795,40.758498,-73.937835,40.758415,208,2010,12,28


In [169]:
# Stats geral das corridas com zero passageiros
zero_passengers = df[df['passenger_count'] == 0]
zero_passengers.describe()

,id,fare,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day
count,7.080000e+02,708.000000,708.000000,708.000000,708.000000,708.000000,708.0,708.000000,708.000000,708.000000
mean,2.675977e+07,9.439266,-71.887276,39.600657,-71.989742,39.658641,0.0,2011.355932,5.899718,15.524011
std,1.596255e+07,6.731690,12.265380,6.756683,11.963165,6.590469,0.0,0.642996,3.554363,8.867610
min,1.371400e+04,2.500000,-74.017300,0.000000,-74.016154,0.000000,0.0,2009.000000,1.000000,1.000000
25%,1.325389e+07,5.300000,-73.992202,40.734526,-73.991022,40.733205,0.0,2011.000000,3.000000,8.000000
50%,2.599671e+07,7.300000,-73.980950,40.754720,-73.979650,40.753200,0.0,2011.000000,5.000000,15.000000
75%,4.053639e+07,11.300000,-73.965799,40.767425,-73.962868,40.769800,0.0,2012.000000,9.000000,23.000000
max,5.531657e+07,57.300000,0.000000,40.833400,0.000000,40.864822,0.0,2015.000000,12.000000,31.000000


## 4.2 - Coordinates

In [170]:
sum(df['pickup_latitude'] == 0)

3782

In [171]:
sum(df['pickup_longitude'] == 0)

3786

In [172]:
sum(df['dropoff_latitude'] == 0)

3758

In [173]:
sum(df['dropoff_longitude'] == 0)

3764

In [174]:
zero_coordenates = df.apply(lambda x: True if all([x['pickup_latitude'] == 0, x['pickup_longitude'] == 0, x['dropoff_latitude'] == 0, x['dropoff_longitude'] == 0]) else False, axis=1)
print(f'Existem {sum(zero_coordenates)} corridas com todas as coordenadas zeradas')

Existem 3587 corridas com todas as coordenadas zeradas


In [175]:
df[zero_coordenates]

,id,fare,pickup_date,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day
7,44195482,2.5,2012-12-11 13:52:00+00:00,0.0,0.0,0.0,0.0,1,2012,12,11
11,6379048,8.5,2011-05-23 22:15:00+00:00,0.0,0.0,0.0,0.0,1,2011,5,23
65,21993993,6.0,2014-05-05 19:27:00+00:00,0.0,0.0,0.0,0.0,1,2014,5,5
92,1454546,6.1,2011-12-02 14:07:00+00:00,0.0,0.0,0.0,0.0,1,2011,12,2
120,17358122,6.5,2010-08-20 19:39:48+00:00,0.0,0.0,0.0,0.0,1,2010,8,20
...,...,...,...,...,...,...,...,...,...,...,...
199718,49162936,4.9,2011-08-19 07:16:00+00:00,0.0,0.0,0.0,0.0,1,2011,8,19
199724,46007628,7.5,2013-10-11 11:25:41+00:00,0.0,0.0,0.0,0.0,1,2013,10,11
199880,35013970,6.5,2014-02-22 06:45:46+00:00,0.0,0.0,0.0,0.0,1,2014,2,22
199883,44115598,12.5,2012-09-10 17:39:00+00:00,0.0,0.0,0.0,0.0,2,2012,9,10


In [176]:
df1 = df.copy()
df1 = df1[(df1['pickup_latitude'] != 0) & (df1['pickup_longitude'] != 0)]
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 196210 entries, 0 to 199999
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   id                 196210 non-null  int64              
 1   fare               196210 non-null  float64            
 2   pickup_date        196210 non-null  datetime64[ns, UTC]
 3   pickup_longitude   196210 non-null  float64            
 4   pickup_latitude    196210 non-null  float64            
 5   dropoff_longitude  196210 non-null  float64            
 6   dropoff_latitude   196210 non-null  float64            
 7   passenger_count    196210 non-null  int64              
 8   year               196210 non-null  int64              
 9   month              196210 non-null  int64              
 10  day                196210 non-null  int64              
dtypes: datetime64[ns, UTC](1), float64(5), int64(5)
memory usage: 18.0 MB


In [177]:
df1 = df1[(df1['dropoff_latitude'] != 0) & (df1['dropoff_longitude'] != 0)]
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 196031 entries, 0 to 199999
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   id                 196031 non-null  int64              
 1   fare               196031 non-null  float64            
 2   pickup_date        196031 non-null  datetime64[ns, UTC]
 3   pickup_longitude   196031 non-null  float64            
 4   pickup_latitude    196031 non-null  float64            
 5   dropoff_longitude  196031 non-null  float64            
 6   dropoff_latitude   196031 non-null  float64            
 7   passenger_count    196031 non-null  int64              
 8   year               196031 non-null  int64              
 9   month              196031 non-null  int64              
 10  day                196031 non-null  int64              
dtypes: datetime64[ns, UTC](1), float64(5), int64(5)
memory usage: 17.9 MB


In [179]:

df1.describe()

,id,fare,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day
count,1.960310e+05,196031.000000,196031.000000,196031.000000,196031.000000,196031.000000,196031.000000,196031.000000,196031.000000,196031.000000
mean,2.771193e+07,11.343009,-73.930055,40.705591,-73.917108,40.688590,1.685228,2011.739541,6.282272,15.705271
std,1.600944e+07,9.799572,5.449368,5.426844,8.518210,3.994266,1.388150,1.858878,3.439841,8.688299
min,1.000000e+00,-52.000000,-1340.648410,-74.015515,-3356.666300,-881.985513,0.000000,2009.000000,1.000000,1.000000
25%,1.383107e+07,6.000000,-73.992268,40.736396,-73.991592,40.735265,1.000000,2010.000000,3.000000,8.000000
50%,2.774864e+07,8.500000,-73.982101,40.753287,-73.980521,40.753726,1.000000,2012.000000,6.000000,16.000000
75%,4.154481e+07,12.500000,-73.968313,40.767542,-73.965316,40.768326,2.000000,2013.000000,9.000000,23.000000
max,5.542357e+07,499.000000,57.418457,1644.421482,1153.572603,872.697628,208.000000,2015.000000,12.000000,31.000000


In [181]:
df1.columns.tolist()

['id',
 'fare',
 'pickup_date',
 'pickup_longitude',
 'pickup_latitude',
 'dropoff_longitude',
 'dropoff_latitude',
 'passenger_count',
 'year',
 'month',
 'day']

In [185]:
# Distancia das corridas
coord_cols = ['pickup_longitude',
 'pickup_latitude',
 'dropoff_longitude',
 'dropoff_latitude']

df1['distance'] = df1.loc[:,coord_cols].apply(lambda x: haversine( (x['pickup_latitude'], x['pickup_longitude']), 
                                                                         (x['dropoff_latitude'], x['dropoff_longitude']) ),axis=1)

ValueError: Longitude -748.016667 is out of range [-180, 180]